In [1]:
%run Evaluation-lib.ipynb
import spot

# Prepare automata
 * Unpack the automata in '.ats' format as created by Ultimate Automizer during the termination
    analysis.
 * Convert the automata into the `.hoa` format for `spot` and `.gff` format for `GOAL`.

In [2]:
if not os.path.exists("semideterministic"):
    print("Unpacking semideterministic.tar.xz")
    !tar -xf semideterministic.tar.xz
if not os.path.exists("semideterministic"):
    print("Failed too unpack the source automata")

In [3]:
tmp = !ls semideterministic/*.ats
names = [os.path.splitext(aut)[0] for aut in tmp]

for aut in names:
    convert_ats_to_hoa_gff(aut)

# Generate Makefile
We create up to four additional subdirctories:
 * `complement` -- stores the complemented automata used for comparison, i.e. produced by GOAL and Ultimate
 * `simplified` -- stores the simplified automata
 * `ltl2dstar` -- stores the complements used for verification, built by `ltl2dstar`
 * `products` -- stores all products needed to test correctness
 
Each subdirectory has its own `Makefile` and the top-level `Makefile` only runs them recursively and set proper dependencies. We use the simplified (by Spot's `autfilt`) automata for computing complement by `ltl2dstar` in order to achieve results for more complements.

The process of creating `Makefile` for a subdirectory (`target`) is always splitted into two halves:
1. `target` : `files`
2. rules for each file

## Complements by GOAL and Ultimate

In [4]:
def complement_target():
    complements = []
    for case in get_cases():
        complements.extend(get_complement_names(case))
    return 'complement: {}\n\t\n'.format(' '.join(complements))

def complement_GOAL_rule(aut,alg,timeout='5m'):
    data = {
        'inputFile' : '../semideterministic/{}.gff'.format(aut),
        'outputFile' : '{}__ComplementGoal{}.hoa'.format(aut,alg['name']),
        'timelimit' : timeout
    }
    data.update(alg)
    command = "\
timeout {timelimit} ../GOAL-20151018/gc batch \'$$compl = complement \
-m {args} {inputFile};save -c HOAF $$compl \
{outputFile};\'"
    cmd = command.format(**data)
    rule = '{outputFile} : {inputFile}\n'.format(**data)
    rule += '\tJVMARGS=-Xmx8g ; '
    rule += '\ttouch {1} && {0} || touch {1}\n'.format(cmd, data['outputFile'])
    return rule

def complement_Ulti_rule(aut,timeout='5m'):
    data = {
        'inputFile' : '../semideterministic/{}.ats'.format(aut),
        'outputFile' : '{}__ComplementUltimateBS'.format(aut),
        'tmpFile' : 'tmp_{}.ats'.format(aut),
        'timelimit' : timeout
    }
    
    script = \
'parseAutomata("{inputFile}");\\n\
NestedWordAutomaton complement = buchiComplementNCSB(nwa);\\n\
write(complement, "{outputFile}", "HOA");'
    #with open(data['tmpFile'],'w') as tmp_f:
    #    print(script.format(**data),file=tmp_f)
    # TADY CHCI POUZIT ECHO
    rule = "{outputFile}.hoa : {inputFile}\n".format(**data)
    rule += '\tJVMARGS=-Xmx8g ; '
    rule += 'echo \'{0}\' > {1}\n'.format(script.format(**data),data['tmpFile'])
    cmd = 'timeout {timelimit} ../Ultimate/Ultimate \
--console ../Ultimate/AutomataScriptInterpreter.xml {tmpFile}'.format(**data)
    sedcmd = 'sed -i "/AP/,/Acceptance/ s/p\([0-9]\+\)/\\1/g" {outputFile}.hoa'.format(**data)
    rule += '\ttouch {2}.hoa && {0} && {1} || touch {2}.hoa\n'.format(cmd, sedcmd, data['outputFile'])
    rule += '\trm -f {tmpFile}\n'.format(**data)
    return rule

## Simplifications

In [5]:
def simplify_rule(aut,degree='low',timeout='5m'):
    rule='{0} : {1}\n'.format(get_simplified_name(aut,degree),\
                                 os.path.join(os.pardir,aut))
    rule+='\t{1} || touch {0}\n'.format(get_simplified_name(aut,degree),
            get_simplify_command(os.path.join(os.pardir,aut),\
            get_simplified_name(aut,degree),degree,timeout))
    return rule

def simplified_target(degree='low'):
    simp_paths = []
    for case in get_cases():
        simp_paths.extend([get_simplified_name(aut,degree) for aut in get_complements(case)])
    return 'simp_{0}: {1}\n\t\n'.format(degree,' '.join(simp_paths))

## Correctness definitions

In order to verify whether $C$ is a complement of $A$ we have to check:

1. $L(A)\cap L(C) = \emptyset$
2. $\overline{L(A)}\cap \overline{L(C)} = \emptyset$

The first is easily done by `autfilt`, for the other we need the complements of $A$ and $C$ performed by LTL2DSTAR. Since LTL2DSTAR works with propositional alphabet and we work with letter alphabet, we have to adeal with this trouble. It can be done by making a product with **validate automaton**, which enforces that exactly one AP is valid at all times.

The workflow is that we create the complements of all automata, and the corresponding products by `autfilt` parallely by using Makefile. We use the simplified automata so that more complements can be computed.

### Ltl2dstar complements

In [6]:
def ltl2dstar_target():
    automata = []
    for case in get_cases():
        automata.append(get_orig_automaton(case))
        automata.extend(get_complements(case))
    return 'ltl2dstar: {}\n\t\n'.format(' '.join([get_ltl2dstar_name(aut) for aut in automata]))

def ltl2dstar_rule(aut,timeout='5m'):
    comp_filename = get_ltl2dstar_name(aut)
    if get_algorithm_name(aut) == 'input':
        source = par(aut)
    else:
        source = par(get_simplified_path(aut,'high'))
    cmd = 'timeout {} ltl2dstar -B -H --complement-input=yes {} - | \
autfilt -H --merge-transitions > {}'.format(timeout,source,comp_filename)
    return '{0}: {1}\n\t{2} || touch {0}\n'.format(comp_filename,source,cmd)

### Commands for validate automata

In [7]:
def get_aps_for_case(case):
    a = spot.automaton(get_orig_automaton(case))
    return [str(ap) for ap in a.ap()]

In [8]:
def create_validate_automaton_cmd(case,timeout='5m'):
    aps = get_aps_for_case(case)
    return 'timeout {1} ltl2tgba -H -f \'G({0})\''.format("|".join(aps),timeout) +\
    ' | timeout {1} autfilt -H --exclusive-ap={0}'.format(",".join(aps),timeout)

### Commands for the `products` target in `Makefile`

In [9]:
def get_products_target():
    complements = []
    for case in get_cases():
        complements.extend(get_complements(case))
    prods = ['prod_{}'.format(os.path.basename(c)) for c in complements]
    comp_prods = ['comp_prod_{}'.format(os.path.basename(c)) for c in complements]
    return 'prod: {} {}\n\t'.format(' '.join(prods),' '.join(comp_prods))

In [10]:
def get_prod_rules(case,timeout='5m'):
    rules = '{0} : {2}\n\t{1} > {0} || touch {0}\n'.\
        format(get_validate_name(case),\
               create_validate_automaton_cmd(case),\
               par(get_orig_automaton(case)))
    for complement in get_complements(case):
        pr_complement=os.path.basename(complement)
        paths=dict()
        paths['product']='prod_{}'.format(pr_complement)
        paths['comp_product']='comp_prod_{}'.format(pr_complement)
        paths['A']=par(get_orig_automaton(case))
        paths['C']=par(complement)
        paths['comp_A']=par(get_ltl2dstar_path(get_orig_automaton(case)))
        paths['comp_C']=par(get_ltl2dstar_path(complement))
        paths['validate']=get_validate_name(case)
        paths['timeout']='timeout {}'.format(timeout)
        rules+='''
{comp_product} : {comp_A} {comp_C} {validate}
\t{timeout} autfilt --product={validate} -H {comp_A} |\
  {timeout} autfilt --product={comp_C} -H > {comp_product} ||\
  touch {comp_product}
    
{product} : {A} {C}
\t{timeout} autfilt --product={C} -H {A} > {product} || touch {product}
    '''.format(**paths)
    return rules

### Creation of `Makefile` files

Run only the two following cells and ignore the rest in case you do not want to check correctness by `ltl2dstar` complementation (to Streett automata).

In [11]:
Makefile=open('Makefile','w')
mktext='''\
SUBDIRS = simplified complement

.PHONY: subdirs $(SUBDIRS)

subdirs: $(SUBDIRS)

$(SUBDIRS):
\t$(MAKE) -C $@
'''
print(mktext,file=Makefile)
Makefile.close()

####  `Makefile` for complements

In [12]:
os.makedirs('complement',exist_ok=True)
lm = open(os.path.join('complement','Makefile'),'w')
print(complement_target(),file=lm)
for aut in get_cases():
    print(complement_Ulti_rule(aut),file=lm)
    for alg in GOAL_algorithms:
        print(complement_GOAL_rule(aut,alg),file=lm)
lm.close()

#### `Makefile` for simplifications

In [13]:
os.makedirs('simplified',exist_ok=True)
sm = open(os.path.join('simplified','Makefile'),'w')
print(simplified_target('high'),file=sm)
for case in get_cases():
    for aut in get_complements(case):
        print(simplify_rule(aut,'high'),file=sm)
sm.close()

### Makefiles for correctness checking

Creation of the complements and especially products can take a lot of time. Do not run these cells if you do not want to make the correctness check.

In [14]:
Makefile=open('Makefile','w')
mktext='''\
SUBDIRS = products ltl2dstar simplified complement

.PHONY: subdirs $(SUBDIRS)

subdirs: $(SUBDIRS)

$(SUBDIRS):
\t$(MAKE) -C $@

ltl2dstar: simplified

products: ltl2dstar

'''
print(mktext,file=Makefile)
Makefile.close()

#### `Makefile` for complements by `ltl2dstar`

In [15]:
ltlstar_to = '5m'
os.makedirs('ltl2dstar',exist_ok=True)
lm = open(os.path.join('ltl2dstar','Makefile'),'w')
print(ltl2dstar_target(),file=lm)
for case in get_cases():
    print(ltl2dstar_rule(get_orig_automaton(case),ltlstar_to),file=lm)
    for aut in get_complements(case):
        print(ltl2dstar_rule(aut,ltlstar_to),file=lm)
lm.close()

#### `Makefile` for products

In [16]:
os.makedirs('products',exist_ok=True)
pm = open(os.path.join('products','Makefile'),'w')
print(get_products_target(),file=pm)
for case in get_cases():
    print(get_prod_rules(case),file=pm)
pm.close()

### Wrapper functions
These can be used outside this notebook to control which subtargets will be created.

In [17]:
def Makefile_only_simplified():
    Makefile=open('Makefile','w')
    mktext='''\
SUBDIRS = simplified complement

.PHONY: subdirs $(SUBDIRS)

subdirs: $(SUBDIRS)

$(SUBDIRS):
\t$(MAKE) -C $@
'''
    print(mktext,file=Makefile)
    Makefile.close()

In [18]:
def Makefile_all():
    Makefile=open('Makefile','w')
    mktext='''\
SUBDIRS = products ltl2dstar simplified complement

.PHONY: subdirs $(SUBDIRS)

subdirs: $(SUBDIRS)

$(SUBDIRS):
\t$(MAKE) -C $@

ltl2dstar: simplified

products: ltl2dstar

'''
    print(mktext,file=Makefile)
    Makefile.close()